# 2022-01-06_ocr_annotated_fix

this script will take the OCR ANNOTATION XLSX file and hydrate it to add all the correct annotations
end result will the clean dataset


* input: id of the DocumentAnalysis objects.
* output: all the fields of DocumentResult as columns. 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important;}</style>'))
 

In [41]:
import datetime

In [42]:
DATA_DIR=Path("../data/interim/2022-01-06_ocr_annotated")
!tree $DATA_DIR

../data/interim/2022-01-06_ocr_annotated
├── DocumentAnalysis-2022-01-30_ktp_annotated.csv
├── DocumentAnalysis-2022-01-30_ktp_cleaned.csv
├── DocumentAnalysis-2022-01-30_ktp.csv
├── DocumentAnalysis-2022-01-30_ktp_inference.csv
├── DocumentAnalysis-2022-01-30_ktp_metrics.csv
├── DocumentAnalysis-2022-01-30_sim_annotated.csv
├── DocumentAnalysis-2022-01-30_sim_cleaned.csv
├── DocumentAnalysis-2022-01-30_sim.csv
├── DocumentAnalysis-2022-01-30_sim_inference.csv
├── DocumentAnalysis-2022-01-30_stnk_annotated.csv
├── DocumentAnalysis-2022-01-30_stnk_cleaned.csv
├── DocumentAnalysis-2022-01-30_stnk.csv
├── DocumentAnalysis-2022-01-30_stnk_inference.csv
├── ocr_annotation_tracking_cleaned.csv
└── OCR ANNOTATION TRACKING.xlsx

0 directories, 15 files


we will need to generate a score for each of the documents.

1. compare relevant fields using a scorer
2. generate result into a csv

In [43]:
!pip install openpyxl

In [44]:
annot = pd.read_excel(DATA_DIR/"OCR ANNOTATION TRACKING.xlsx", names=['Unnamed: 0', 'image', 'is_colored', 'file_type', 'gcs_link', 'analysis_id', 'is_invalid', 'annotation_error', 'error2'])
annot.head()

,Unnamed: 0,image,is_colored,file_type,gcs_link,analysis_id,is_invalid,annotation_error,error2
0,0,202106100907_0a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,528.0,1.0,NaN,NaN
1,1,202106100907_13a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,529.0,NaN,NaN,NaN
2,2,202106100907_5.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,533.0,1.0,NaN,NaN
3,3,202106100909_2.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,534.0,NaN,NaN,NaN
4,4,202106100912_12a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,536.0,NaN,NaN,NaN


In [45]:
annot.columns

Index(['Unnamed: 0', 'image', 'is_colored', 'file_type', 'gcs_link',
       'analysis_id', 'is_invalid', 'annotation_error', 'error2'],
      dtype='object')

In [46]:
annot

,Unnamed: 0,image,is_colored,file_type,gcs_link,analysis_id,is_invalid,annotation_error,error2
0,0,202106100907_0a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,528.0,1.0,NaN,NaN
1,1,202106100907_13a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,529.0,NaN,NaN,NaN
2,2,202106100907_5.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,533.0,1.0,NaN,NaN
3,3,202106100909_2.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,534.0,NaN,NaN,NaN
4,4,202106100912_12a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,536.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
764,764,All KYC Docs_page-0187_denoising_bg_shadow_out...,False,STNK,gs://som-datasets/interim/2022-01-06_combined_...,NaN,1.0,413 Request Entity Too Large,NaN
765,765,KTP23.webp,True,KTP,gs://som-datasets/interim/2022-01-06_combined_...,573.0,NaN,NaN,NaN
766,766,DL30.webp,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,NaN,NaN,NaN,NaN
767,767,DL39.webp,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,NaN,NaN,NaN,NaN


In [47]:
annot.columns

Index(['Unnamed: 0', 'image', 'is_colored', 'file_type', 'gcs_link',
       'analysis_id', 'is_invalid', 'annotation_error', 'error2'],
      dtype='object')

In [48]:
annot = annot[['image', 'is_colored', 'file_type', 'gcs_link', 'analysis_id',
       'is_invalid', 'annotation_error']].copy()
annot['is_invalid'] = annot['is_invalid'].fillna('0').astype(int)
annot['annotation_error'] = annot['annotation_error'].fillna('')
annot['analysis_id'] = annot['analysis_id'].fillna('0').astype(int)
annot.head()

,image,is_colored,file_type,gcs_link,analysis_id,is_invalid,annotation_error
0,202106100907_0a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,528,1,
1,202106100907_13a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,529,0,
2,202106100907_5.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,533,1,
3,202106100909_2.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,534,0,
4,202106100912_12a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,536,0,


In [49]:
annot.to_csv(DATA_DIR/"ocr_annotation_tracking_cleaned.csv")

lets split into ktp, stnk, sim for processing by script.

In [50]:
annot.file_type.value_counts()

STNK       379
SIM-OLD    247
KTP        135
SIM-NEW      8
Name: file_type, dtype: int64

In [53]:
annot['doc'] = annot['file_type'].apply(lambda x: {"ktp":"ktp", "stnk":"stnk", "sim-old": "sim", "sim-new": "sim"}[x.lower()].upper())
annot.doc.value_counts()

STNK    379
SIM     255
KTP     135
Name: doc, dtype: int64

In [55]:
annot[annot.doc == 'KTP'].to_csv(DATA_DIR/"ocr_annotation_tracking_cleaned_ktp.csv")
annot[annot.doc == 'STNK'].to_csv(DATA_DIR/"ocr_annotation_tracking_cleaned_stnk.csv")
annot[annot.doc == 'SIM'].to_csv(DATA_DIR/"ocr_annotation_tracking_cleaned_sim.csv")

In [56]:
annot

,image,is_colored,file_type,gcs_link,analysis_id,is_invalid,annotation_error,doc
0,202106100907_0a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,528,1,,KTP
1,202106100907_13a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,529,0,,KTP
2,202106100907_5.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,533,1,,KTP
3,202106100909_2.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,534,0,,KTP
4,202106100912_12a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,536,0,,KTP
...,...,...,...,...,...,...,...,...
764,All KYC Docs_page-0187_denoising_bg_shadow_out...,False,STNK,gs://som-datasets/interim/2022-01-06_combined_...,0,1,413 Request Entity Too Large,STNK
765,KTP23.webp,True,KTP,gs://som-datasets/interim/2022-01-06_combined_...,573,0,,KTP
766,DL30.webp,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM
767,DL39.webp,True,SIM-OLD,gs://som-datasets/interim/2022-01-06_combined_...,0,0,,SIM


# Second

In [84]:
df = pd.read_csv(DATA_DIR/"DocumentAnalysis-2022-01-30_ktp.csv", 
                 parse_dates=['created_at'])
df.head()

,id,internal_album,internal_image_id,url,image,image_id,type,requestor,is_completed,result,created_at
0,702,NaN,NaN,https://about.lovia.life/wp-content/uploads/20...,NaN,NaN,KTP,NaN,0,NaN,2022-01-18 05:34:57
1,701,NaN,NaN,https://about.lovia.life/wp-content/uploads/20...,NaN,NaN,KTP,NaN,0,NaN,2022-01-18 05:12:36
2,700,NaN,NaN,https://about.lovia.life/wp-content/uploads/20...,NaN,NaN,KTP,NaN,0,NaN,2022-01-18 05:05:14
3,699,NaN,NaN,https://about.lovia.life/wp-content/uploads/20...,NaN,NaN,KTP,NaN,0,NaN,2022-01-18 04:24:43
4,618,NaN,NaN,NaN,ocr/analysis/KTP34.jpeg,NaN,KTP,NaN,1,414.0,2022-01-09 15:54:51


In [85]:
df = df[~df['result'].isnull()]
df['result'] = df['result'].astype(int)

In [86]:
dres = pd.merge(df, annot, on='id')
dres.head()

,id,internal_album,internal_image_id,url,image,image_id,type,requestor,is_completed,result,created_at,image_file_name,is_colored,file_type,gcs_link,is_invalid
0,617,NaN,NaN,NaN,ocr/analysis/KTP27.jpeg,NaN,KTP,NaN,1,413,2022-01-09 15:33:47,KTP27.jpeg,True,KTP,gs://som-datasets/interim/2022-01-06_combined_...,NaN
1,616,NaN,NaN,NaN,ocr/analysis/KTP11.jpeg,NaN,KTP,NaN,1,412,2022-01-09 15:29:18,KTP11.jpeg,True,KTP,gs://som-datasets/interim/2022-01-06_combined_...,NaN
2,615,NaN,NaN,NaN,ocr/analysis/KTP12.jpeg,NaN,KTP,NaN,1,411,2022-01-09 15:24:54,KTP12.jpeg,True,KTP,gs://som-datasets/interim/2022-01-06_combined_...,NaN
3,614,NaN,NaN,NaN,ocr/analysis/KTP13.jpeg,NaN,KTP,NaN,1,410,2022-01-09 15:19:37,KTP13.jpeg,True,KTP,gs://som-datasets/interim/2022-01-06_combined_...,NaN
4,613,NaN,NaN,NaN,ocr/analysis/KTP14.jpeg,NaN,KTP,NaN,1,409,2022-01-09 15:05:22,KTP14.jpeg,True,KTP,gs://som-datasets/interim/2022-01-06_combined_...,NaN


In [87]:
dres = dres[['id', 'image', 'gcs_link', 'type', 'is_completed', 'created_at', 'result']]

In [88]:
dres.to_csv(DATA_DIR/"DocumentAnalysis-2022-01-30_ktp_cleaned.csv", index=False)

run for other docs

In [89]:
df = pd.read_csv(DATA_DIR/"DocumentAnalysis-2022-01-30_stnk.csv", 
                 parse_dates=['created_at'])
df = df[~df['result'].isnull()]
df['result'] = df['result'].astype(int)
dres = pd.merge(df, annot, on='id')
dres.head()
dres = dres[['id', 'image', 'gcs_link', 'type', 'is_completed', 'created_at', 'result']]
dres.to_csv(DATA_DIR/"DocumentAnalysis-2022-01-30_stnk_cleaned.csv", index=False)

In [90]:
df = pd.read_csv(DATA_DIR/"DocumentAnalysis-2022-01-30_sim.csv", 
                 parse_dates=['created_at'])
df = df[~df['result'].isnull()]
df['result'] = df['result'].astype(int)
dres = pd.merge(df, annot, on='id')
dres.head()
dres = dres[['id', 'image', 'gcs_link', 'type', 'is_completed', 'created_at', 'result']]
dres.to_csv(DATA_DIR/"DocumentAnalysis-2022-01-30_sim_cleaned.csv", index=False)

In [ ]:
file:///home/darshansonde/Code/work/2021_tolaram/insureka-ml/ocr/data/interim/2022-01-06_ocr_annotated/DocumentAnalysis-2022-01-30_stnk_cleaned.csv

In [ ]:
'agama',
 'alamat',
 'berlaku_hingga',
 'jenis_kelamin',
 'kecamatan',
 'kel_desa',
 'kewarganegaraan',
 'nama',
 'nik',
 'pekerjaan',
 'rt_rw',
 'status_perkawinan',
 'tempat',
 'tgl_lahir'

# Third

Take the KTP document and remove invalid from the cleaned fiels

In [19]:
df = pd.read_csv("/home/darshansonde/Code/work/2021_tolaram/insureka-ml/ocr/data/interim/2022-01-06_combined_orc_kyc/ktp_annotated.csv")

In [20]:
df.shape

(135, 24)

In [21]:
df.head()

,Unnamed: 0,Unnamed: 0.1,image,is_colored,file_type,gcs_link,analysis_id,is_invalid,annotation_error,doc,...,kecamatan,kel_desa,kewarganegaraan,nama,nik,pekerjaan,rt_rw,status_perkawinan,tempat,tgl_lahir
0,0,0,202106100907_0a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,528,1,NaN,KTP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,202106100907_13a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,529,0,NaN,KTP,...,BATAM KOTA,SUNGAI PANAS,WNI,KASTODI,2171060103890002,WIRASWASTA,003/009,KAWIN,BATAM,01-03-1989
2,2,2,202106100907_5.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,533,1,NaN,KTP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,202106100909_2.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,534,0,NaN,KTP,...,NONOBA,NaN,WNI,DEDY,2171040509849001,WIRASWASTA,001 /003,BELUM KAWIN,PULAII TODAX,05-09-1954
4,4,4,202106100912_12a.jpg,False,KTP,gs://som-datasets/interim/2022-01-06_combined_...,536,0,NaN,KTP,...,PERHENTIAN RAJA,PANTAI RAJA,WNI,WAHYUDI HARAHAP,1401160403920001,WIRASWASTA,002/004,KAWIN,PANTAI RAJA,04-03-1992


In [22]:
df = df[['image', 'is_colored', 'file_type',
       'gcs_link', 'analysis_id', 'is_invalid', 'annotation_error', 'doc',
       'agama', 'alamat', 'berlaku_hingga', 'jenis_kelamin', 'kecamatan',
       'kel_desa', 'kewarganegaraan', 'nama', 'nik', 'pekerjaan', 'rt_rw',
       'status_perkawinan', 'tempat', 'tgl_lahir']].copy()

In [23]:
df['file'] = df['image']

In [24]:

df = df[~((df['file'] == '202106100904_0.jpg') | (df['file'] == '202106100904_16.jpg')| (df['file'] == 'STNK9.jpeg')) ] #stnk
df = df[~(df['file'].str.contains('denoising') | (df['file'] == 'All_KYC_Docs_50.jpg'))] #blurry docs
df = df[~(df['file'] == '202106100907_0a.jpg')]

#manual inspection removing invalid ones 
df = df[~(df['file'].isin(['202106100909_2.jpg', '202106100912_12a.jpg', '202106100918_0a.jpg', 
                 '202106100918_4.jpg', '202106100918_9a.jpg', '202106100923_1a.jpg', 
                 '202106100924_0.jpg', '202106100924_3a.jpg', '202106100925_0.jpg', 
                 'All_KYC_Docs_105b.jpg', 'All_KYC_Docs_122b.jpg', 'All_KYC_Docs_127a.jpg', 
                 'All_KYC_Docs_127.jpg', 'All_KYC_Docs_130.jpg', 'All_KYC_Docs_14b.jpg', 
                 'All_KYC_Docs_151.jpg', 'All_KYC_Docs_156b.jpg', 'All_KYC_Docs_171.jpg', 
                 'All_KYC_Docs_182.jpg', 'All_KYC_Docs_185a.jpg', 'All_KYC_Docs_194a.jpg', 
                 'All_KYC_Docs_45a.jpg', 'All_KYC_Docs_45.jpg', 'All_KYC_Docs_58a.jpg', 
                 'All_KYC_Docs_90b.jpg', 'All_KYC_Docs_99c.jpg', 'Copy_STNK_&_KTP_10.jpg', 
                 'Copy_STNK_&_KTP_19a.jpg', 'Copy_STNK_&_KTP_2a.jpg', 'Copy_STNK_&_KTP_4b.jpg', 
                 'Copy_STNK_&_KTP_6a.jpg', 'Copy_STNK_&_KTP_6.jpg' , 'All_KYC_Docs_115a.jpg', 
                '202106100907_5.jpg', 'KTP35.jpeg', 'KTP1.jpeg', 'KTP34.jpeg', 'KTP33.jpeg', 'KTP36.jpg']))]


In [25]:
df.shape

(53, 23)

In [ ]:
df.to_csv()